In [16]:
using PyCall
nx = pyimport("networkx")
np = pyimport("numpy")
alg = pyimport("numpy.linalg")
fun = pyimport("functools")

#G = nx.random_geometric_graph(10, 0.2, dim=2, p=2)

G = nx.DiGraph([(0,1),(0,2),(0,3),(1,2),(1,3),(2,3),(2,4)]) #network 
println(G.edges())
#for i in G.predecessors(2)
    #println(i)
#end

# initial state
#state_vector =[0,0,1,1]  #ones(Int,length(G.nodes()))


PyObject OutEdgeView([(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3), (2, 4)])


In [19]:
chain =[0,4,2] 
path = nx.shortest_path(G, chain[1], chain[2])
2 in path

true

In [ ]:
vtx_list = collect(G.nodes())    
active_nodes_list = zeros(Int,0)  
new_state_vector = zeros(Int,0)

for vtx in vtx_list
    vtx_index = indexin(vtx,vtx_list)[1] 
    println(vtx_index)
    predecessors = collect(G.predecessors(vtx))
    println(predecessors)

    if length(predecessors) == 0
        println("case 1")
        updated_state_vtx = state_vector[vtx_index] 
    elseif length(predecessors) == 1
        println("case 2")
        index_pred = indexin(predecessors[1],vtx_list)[1] 
        updated_state_vtx = xor(state_vector[vtx_index],state_vector[index_pred])
    elseif length(predecessors) > 1
        println("case 3")
        values = map(x->state_vector[indexin(x,vtx_list)[1]],predecessors)
        xor_pred = fun.reduce(xor, values)
        updated_state_vtx = xor(state_vector[vtx_index],xor_pred)
    end

    println("updated state for vtx ",vtx," is ",updated_state_vtx)

    if updated_state_vtx == 1
        append!(active_nodes_list,vtx) 
    end
    #println(active_nodes_list)
    append!(new_state_vector,updated_state_vtx)
end 

active_subgraph = nx.induced_subgraph(G, active_nodes_list)
println(new_state_vector,active_nodes_list,active_subgraph.edges.())

In [ ]:
#define activation function as xor of incoming nodes

active_nodes_list = zeros(Int,0)  
new_state_vector = zeros(Int,0)
for vtx in collect(G.nodes())
    #println(vtx)
    predecessors = collect(G.predecessors(vtx))
    #println(predecessors)

    if length(predecessors) == 0
        state_updated_vtx = state_vector[vtx] 
    elseif length(predecessors) == 1
        state_updated_vtx = xor(Int(state_vector[vtx]),Int(state_vector[predecessors[1]]))
    elseif length(predecessors) > 1
        xor_pred = fun.reduce((a, b)->xor(Int(state_vector[a]),Int(state_vector[b])), predecessors)
        state_updated_vtx = xor(Int(state_vector[vtx]),xor_pred)
    end

    #println(vtx,",",state_updated_vtx)

    if state_updated_vtx == 1
        append!(active_nodes_list,vtx) 
    end
    #println(active_nodes_list)
    append!(new_state_vector,state_updated_vtx)
end 

println(new_state_vector)

active_subgraph = nx.induced_subgraph(G, active_nodes_list)

In [12]:
## don't really need to change anything for directed graphs

###### SUBSTITE "G" WITH "active_subgraph"

k = 2
l = 3
MC_kl = Array{Int64}[]
vtx=collect(G.nodes())
possible_chains = collect(Iterators.product([vtx for i=1:k+1]...))
chains = [chain for chain in possible_chains if length(chain)==length(Set(chain))]
for chain in chains
    println(chain)
    chain = collect(chain)   
    is_seq = true  
    length = 0
    for i in 1:k
        try
            length = length + nx.shortest_path_length(G, chain[i], chain[i + 1])
        catch
            is_seq = false
        end
        #length = length + nx.shortest_path_length(G, possible_chain[i], possible_chain[i + 1])
        if length > l
            break
        end

        if is_seq == true && nx.shortest_path_length(G, chain[i], chain[i + 1]) > 1 
            subgraph = G.subgraph(chain)
            unfrozen_subgraph = nx.Graph(subgraph)
            if i == k
                if chain[i-1] in unfrozen_subgraph.neighbors(chain[i]) && chain[i-1] in unfrozen_subgraph.neighbors(chain[i+1])
                    is_seq = false
                end
            elseif i == 1
                if chain[i+2] in unfrozen_subgraph.neighbors(chain[i]) && chain[i+2] in unfrozen_subgraph.neighbors(chain[i+1])
                    is_seq = false
                end

            elseif (chain[i+2] in unfrozen_subgraph.neighbors(chain[i]) && chain[i+2] in unfrozen_subgraph.neighbors(chain[i+1])) || (chain[i-1] in unfrozen_subgraph.neighbors(chain[i]) && chain[i-1] in unfrozen_subgraph.neighbors(chain[i+1]))
                is_seq = false
            end
        end

            
    end
        
    if length == l && is_seq == true
        push!(MC_kl,chain)

        #subgraph = G.subgraph(possible_chain)
        #unfrozen_subgraph = nx.Graph(subgraph)
        #unfrozen_subgraph.add_edge(possible_chain[1],possible_chain[k+1])
        #if nx.is_eulerian(unfrozen_subgraph) || unfrozen_subgraph.number_of_edges()==1
        #    push!(MC_kl,possible_chain)
        #end
    end
    #end                                 
end
sort!(MC_kl)
println(length(MC_kl))
                


(2, 1, 0)
(3, 1, 0)
(4, 1, 0)
(1, 2, 0)
(3, 2, 0)
(4, 2, 0)
(1, 3, 0)
(2, 3, 0)
(4, 3, 0)
(1, 4, 0)
(2, 4, 0)
(3, 4, 0)
(2, 0, 1)
(3, 0, 1)
(4, 0, 1)
(0, 2, 1)
(3, 2, 1)
(4, 2, 1)
(0, 3, 1)
(2, 3, 1)
(4, 3, 1)
(0, 4, 1)
(2, 4, 1)
(3, 4, 1)
(1, 0, 2)
(3, 0, 2)
(4, 0, 2)
(0, 1, 2)
(3, 1, 2)
(4, 1, 2)
(0, 3, 2)
(1, 3, 2)
(4, 3, 2)
(0, 4, 2)
(1, 4, 2)
(3, 4, 2)
(1, 0, 3)
(2, 0, 3)
(4, 0, 3)
(0, 1, 3)
(2, 1, 3)
(4, 1, 3)
(0, 2, 3)
(1, 2, 3)
(4, 2, 3)
(0, 4, 3)
(1, 4, 3)
(2, 4, 3)
(1, 0, 4)
(2, 0, 4)
(3, 0, 4)
(0, 1, 4)
(2, 1, 4)
(3, 1, 4)
(0, 2, 4)
(1, 2, 4)
(3, 2, 4)
(0, 3, 4)
(1, 3, 4)
(2, 3, 4)
1


In [15]:
MC_kminus1_l = Array{Int64}[]  
possible_chains = collect(Iterators.product([vtx for i=1:k]...))
chains = [chain for chain in possible_chains if length(chain)==length(Set(chain))]
for chain in chains
    println(chain)
    chain = collect(chain)   
    is_seq = true  
    length = 0
    for i in 1:k-1
        try
            length = length + nx.shortest_path_length(G, chain[i], chain[i + 1])
        catch
            is_seq = false
        end
        #length = length + nx.shortest_path_length(G, possible_chain[i], possible_chain[i + 1])
        if length > l
            break
        end
            
    end
        
    if length == l && is_seq == true
        push!(MC_kminus1_l,chain)

        #subgraph = G.subgraph(possible_chain)
        #unfrozen_subgraph = nx.Graph(subgraph)
        #unfrozen_subgraph.add_edge(possible_chain[1],possible_chain[k])
        #if nx.is_eulerian(unfrozen_subgraph) || unfrozen_subgraph.number_of_edges()==1
        #    push!(MC_kl,possible_chain)
        #end
    end
    #end                                 
end
sort!(MC_kminus1_l)
println(length(MC_kminus1_l))

(1, 0)
(2, 0)
(3, 0)
(4, 0)
(0, 1)
(2, 1)
(3, 1)
(4, 1)
(0, 2)
(1, 2)
(3, 2)
(4, 2)
(0, 3)
(1, 3)
(2, 3)
(4, 3)
(0, 4)
(1, 4)
(2, 4)
(3, 4)
0


In [5]:
MC_kplus1_l = Array{Int64}[] 
for possible_chain in collect(Iterators.product([vtx for i=1:k+2]...))
    possible_chain = collect(possible_chain)   
    is_seq = true
    for i in 1:k+1
        if possible_chain[i] == possible_chain[i+1]
            is_seq = false
        end  
    end
    for i in 2:k+1
        if possible_chain[i-1] == possible_chain[i+1]
            is_seq = false
        end  
    end
    if is_seq == true   
        length = 0
        for i in 1:k+1
            try
                length = length + nx.shortest_path_length(active_subgraph, possible_chain[i], possible_chain[i + 1])
            catch
                is_seq = false
            end

            #length = length + nx.shortest_path_length(G, possible_chain[i], possible_chain[i + 1])
            
            if length > l
                break
            end

            if is_seq == true && nx.shortest_path_length(G, chain[i], chain[i + 1]) > 1 
                subgraph = G.subgraph(chain)
                unfrozen_subgraph = nx.Graph(subgraph)
                if i == k+1
                    if chain[i-1] in unfrozen_subgraph.neighbors(chain[i]) && chain[i-1] in unfrozen_subgraph.neighbors(chain[i+1])
                        is_seq = false
                    end
                elseif i == 1
                    if chain[i+2] in unfrozen_subgraph.neighbors(chain[i]) && chain[i+2] in unfrozen_subgraph.neighbors(chain[i+1])
                        is_seq = false
                    end
    
                elseif (chain[i+2] in unfrozen_subgraph.neighbors(chain[i]) && chain[i+2] in unfrozen_subgraph.neighbors(chain[i+1])) || (chain[i-1] in unfrozen_subgraph.neighbors(chain[i]) && chain[i-1] in unfrozen_subgraph.neighbors(chain[i+1]))
                    is_seq = false
                end
            end
        end
        if length == l && is_seq == true
            push!(MC_kplus1_l,chain)
            #subgraph = G.subgraph(possible_chain)
            #println(subgraph.edges())
            #unfrozen_subgraph = nx.Graph(subgraph)
            #unfrozen_subgraph.add_edge(possible_chain[1],possible_chain[k+2])
            #if nx.is_eulerian(unfrozen_subgraph) || unfrozen_subgraph.number_of_edges()==1
            #    push!(MC_kplus1_l,possible_chain)
            #end
        end
    end                                 
end
println(length(MC_kplus1_l))


0


In [ ]:
row_vec_kl = Array{Int}(undef,0)
col_vec_kl = Array{Int}(undef,0) 
data_kl = Array{Int}(undef,0) 

row_vec_kplus1l = Array{Int}(undef,0)
col_vec_kplus1l = Array{Int}(undef,0) 
data_kplus1l = Array{Int}(undef,0) 

In [ ]:
for k_chain_idx in 1:length(MC_kl)
    k_chain = MC_kl[k_chain_idx]
    is_seq = true
    #println(k_chain)
    for vertex_idx in 2:length(k_chain) - 1

        #if removing a vertex does not change the length of a path
        try
            nx.shortest_path_length(active_subgraph, k_chain[vertex_idx - 1], k_chain[vertex_idx + 1]) == nx.shortest_path_length(active_subgraph,k_chain[vertex_idx - 1],k_chain[vertex_idx]) + nx.shortest_path_length(active_subgraph, k_chain[vertex_idx], k_chain[vertex_idx + 1])
        catch
            is_seq = false
        end

        if is_seq == true
        #if nx.shortest_path_length(G, k_chain[vertex_idx - 1], k_chain[vertex_idx + 1]) == nx.shortest_path_length(G,k_chain[vertex_idx - 1],k_chain[vertex_idx]) + nx.shortest_path_length(G, k_chain[vertex_idx], k_chain[vertex_idx + 1])
            
            #if the k-tuple with the vertex removed is part of MC_{k-1,l}
            kminus1_chain = collect(np.delete(k_chain, vertex_idx-1))
            #println(kminus1_chain)

            if kminus1_chain in MC_kminus1_l
                #println(kminus1_chain)

                #build vectors
                row_index = findfirst(x->x==kminus1_chain,MC_kminus1_l)
                
                push!(row_vec_kl,row_index)
                #println(row_vec_kl,row_index)
                push!(col_vec_kl,k_chain_idx)
                #println(col_vec_kl,k_chain_idx)
                push!(data_kl,(-1)^vertex_idx)
            end
        end
    end
end

In [ ]:
for kplus1_chain_idx in 1:length(MC_kplus1_l)
    kplus1_chain = MC_kplus1_l[kplus1_chain_idx]
    #println(k_chain)
    is_seq = true
    for vertex_idx in 2:length(kplus1_chain) - 1

        #if removing a vertex does not change the length of a path
        try
            nx.shortest_path_length(active_subgraph, kplus1_chain[vertex_idx - 1], kplus1_chain[vertex_idx + 1]) == nx.shortest_path_length(active_subgraph,kplus1_chain[vertex_idx - 1],kplus1_chain[vertex_idx]) + nx.shortest_path_length(active_subgraph, kplus1_chain[vertex_idx], kplus1_chain[vertex_idx + 1])
        catch
            is_seq = false
        end
        
        if is_seq == true
        #if nx.shortest_path_length(G, kplus1_chain[vertex_idx - 1], kplus1_chain[vertex_idx + 1]) == nx.shortest_path_length(G,kplus1_chain[vertex_idx - 1],kplus1_chain[vertex_idx]) + nx.shortest_path_length(G, kplus1_chain[vertex_idx], kplus1_chain[vertex_idx + 1])
        
        #if the k-tuple with the vertex removed is part of MC_{k-1,l}
            k_chain = collect(np.delete(kplus1_chain, vertex_idx-1))
            #println(kminus1_chain)

            if k_chain in MC_kl
                #println(kminus1_chain)

                #build vectors
                row_index = findfirst(x->x==k_chain,MC_kl)
                
                push!(row_vec_kplus1l,row_index)
                #println(row_vec_kplus1l,row_index)
                push!(col_vec_kplus1l,kplus1_chain_idx)
                #println(col_vec_kplus1l,kplus1_chain_idx)
                push!(data_kplus1l,(-1)^vertex_idx)
            end
        end
    end
end

In [ ]:
println(length(row_vec_kl))
println(length(col_vec_kl))
println(length(row_vec_kplus1l))
println(length(col_vec_kplus1l))


In [ ]:
using SparseArrays

#bdry_mtx = sparse(row_vec, col_vec, ones(length(col_vec)))
if length(col_vec_kl) > 0
    cols_of_sparse_kl = last(col_vec_kl)
elseif length(col_vec_kl) == 0
    cols_of_sparse_kl = 0
end


if length(col_vec_kplus1l) > 0
    cols_of_sparse_kplus1l = last(col_vec_kplus1l)
elseif length(col_vec_kplus1l) == 0
    cols_of_sparse_kplus1l = 0
end


if row_vec_kl == Array{Int}(undef,0) && col_vec_kl == Array{Int}(undef,0)
    bdry_mtx_kl = zeros((1, 1))
else
    bdry_mtx_kl = sparse(row_vec_kl, col_vec_kl, data_kl)
end

println(size(bdry_mtx_kl))

if row_vec_kplus1l == Array{Int}(undef,0) && col_vec_kplus1l == Array{Int}(undef,0)
    bdry_mtx_kplus1l = zeros((1, 1))
else
    bdry_mtx_kplus1l = sparse(row_vec_kplus1l, col_vec_kplus1l, data_kplus1l)
end

println(size(bdry_mtx_kplus1l))

In [ ]:
if cols_of_sparse_kl == length(MC_kl)
    dim_kernel = size(bdry_mtx_kl)[2] - alg.matrix_rank(bdry_mtx_kl)
elseif cols_of_sparse_kl == 0
    dim_kernel = length(MC_kl)
else
    entries_to_add = length(MC_kl) - cols_of_sparse_kl
    println(alg.matrix_rank(bdry_mtx_kl))
    dim_kernel = size(bdry_mtx_kl)[2] + entries_to_add - alg.matrix_rank(bdry_mtx_kl)
end
println(dim_kernel)


In [ ]:

if cols_of_sparse_kplus1l == length(MC_kplus1_l)
    dim_image = alg.matrix_rank(bdry_mtx_kplus1l)
else
    entries_to_add = length(MC_kplus1_l) - cols_of_sparse_kplus1l
    dim_image = alg.matrix_rank(bdry_mtx_kplus1l)
end
println(dim_image)

In [ ]:
betti = dim_kernel - dim_image
println("betti_{k,l} for k,l=",k,l," is ", betti)

In [ ]:
a=0
while 1!= 0
    a = a+1
end